### Installing Dependencies

In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

### Mount Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Set-Up

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import warnings
warnings.filterwarnings("ignore")
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import flwr as fl
from flwr.common import Metrics
import torchvision.models as models
import torch.optim as optim

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cuda using PyTorch 2.0.0+cu118 and Flower 1.3.0


## Load Dataset

#### Define Torch Dataloader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = torchvision.datasets.ImageFolder(root='drive/MyDrive/archive/gaussian_filtered_images/gaussian_filtered_images', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

#### Display data

In [ ]:
def index_to_class_name(index, dataset):
    return dataset.classes[index]

len_images = 10
images, labels = next(iter(dataloader))
sample_indices = np.random.choice(len(images), len_images, replace=False)
sample_images = images[sample_indices]
sample_labels = labels[sample_indices]

for i, image in enumerate(sample_images):
    class_name = index_to_class_name(sample_labels[i], dataset)
    plt.subplot(2, 5, i+1)
    plt.imshow(image.permute(1, 2, 0))
    plt.title(class_name)
    plt.axis('off')

plt.show() 

KeyboardInterrupt: ignored

#### Data Splitter Utility

In [ ]:
BATCH_SIZE = 8

In [ ]:
def data_splitter(nodr=[0.2,0.2,0.2,0.2,0.2],mild=[0.2,0.2,0.2,0.2,0.2],moderate=[0.2,0.2,0.2,0.2,0.2],severe=[0.2,0.2,0.2,0.2,0.2],proliferate=[0.2,0.2,0.2,0.2,0.2],num_clients=5):
    datasets_individual=[]
    train_datasets_individual = []
    test_datasets_individual = []
    for i in range(5):
        classes = torch.tensor([i])
        # get indices that correspond to one of the selected classes
        indices = (torch.tensor(dataset.targets)[..., None] == classes).any(-1).nonzero(as_tuple=True)[0]
        data = torch.utils.data.Subset(dataset, indices) # subset the dataset
        datasets_individual.append(data)
    
    for i in range(5):
        train_temp, test_temp = torch.utils.data.random_split(datasets_individual[i], [int(0.8 * len(datasets_individual[i].indices)), len(datasets_individual[i].indices) - int(0.8 * len(datasets_individual[i].indices))])
        train_datasets_individual.append(train_temp)
        test_datasets_individual.append(test_temp)
    
    
    testset = torch.utils.data.ConcatDataset([test_datasets_individual[0],test_datasets_individual[1],test_datasets_individual[2],test_datasets_individual[3],test_datasets_individual[4]])

    train_datasets=[]
    for i in range(5):
        if dataset.classes[i]=="Mild":
            lengths=[a*len(train_datasets_individual[i]) for a in mild]
        elif dataset.classes[i]=="Moderate":
            lengths=[a*len(train_datasets_individual[i]) for a in moderate]
        elif dataset.classes[i]=="No_DR":
            lengths=[a*len(train_datasets_individual[i]) for a in nodr]
        elif dataset.classes[i]=="Proliferate_DR":
            lengths=[a*len(train_datasets_individual[i]) for a in proliferate]
        elif dataset.classes[i]=="Severe":
            lengths=[a*len(train_datasets_individual[i]) for a in severe]
        lengths=[int(l) for l in lengths]
        lengths[4]=len(train_datasets_individual[i])-sum(lengths[0:4])
        train_data = random_split(train_datasets_individual[i],lengths,torch.Generator().manual_seed(42))
        train_datasets.append(train_data)

    client_datasets=[]
    for i in range(num_clients):
        client_datasets.append(torch.utils.data.ConcatDataset([train_datasets[0][i],train_datasets[1][i],train_datasets[2][i],train_datasets[3][i],train_datasets[4][i]]))

    trainloaders = []
    valloaders = []
    test_len = len(testset) // num_clients
    test_lens = [test_len]*num_clients
    test_lens[-1] = len(testset) - test_len*(num_clients-1)
    valset = random_split(testset, test_lens, torch.Generator().manual_seed(42))

    for ds in client_datasets:
      trainloaders.append(DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True))

    for ds in valset:
      valloaders.append(DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True))

    return trainloaders, valloaders


In [ ]:
def split_dataset_to_equal_length_but_unequal_classes(num_clients=5):
    partition_size = len(dataset) // num_clients
    data_points_per_client_lengths = [partition_size]*num_clients
    data_points_per_client_lengths[-1] = len(dataset) - partition_size*(num_clients-1)

    dataset_for_clients = random_split(dataset, data_points_per_client_lengths, torch.Generator().manual_seed(42))

    trainloaders = []
    valloaders = []
    for ds in dataset_for_clients:
        len_val = len(ds) // 5  # 20 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=True))
    
    return trainloaders, valloaders

In [ ]:
def split_dataset_for_global_training():
    trainset, testset= torch.utils.data.random_split(dataset, [2930, 732])
    trainSetLoader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    testSetLoader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)
    return trainSetLoader, testSetLoader

## Model Definition

In [ ]:
# class Net(nn.Module):
#     def __init__(self, num_classes=5):
#         super(Net, self).__init__()
#         self.num_classes = num_classes
#         self.inception = models.googlenet(pretrained=True)
        
#         for param in self.inception.parameters():
#             param.requires_grad = False
            
#         self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)

#     def forward(self, x):
#         x = self.inception(x)
#         return F.log_softmax(x, dim=1)

In [ ]:
# from torchsummary import summary
# vgg = models.vgg16(pretrained=True)
# print(vgg)

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=5):
        super(Net, self).__init__()
        self.num_classes = num_classes
        self.alexnet = models.alexnet(pretrained=True)
        
        for param in self.alexnet.parameters():
            param.requires_grad = False
            
        self.alexnet.classifier[6] = nn.Linear(self.alexnet.classifier[6].in_features, num_classes)

    def forward(self, x):
        x = self.alexnet(x)
        return F.log_softmax(x, dim=1)

In [ ]:
# class Net(nn.Module):
#     def __init__(self) -> None:
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 8, 4)
#         # self.bn1 = nn.BatchNorm2d(8)
#         self.conv2 = nn.Conv2d(8, 16, 4)
#         # self.bn2 = nn.BatchNorm2d(16)
#         self.conv3 = nn.Conv2d(16,32,4)
#         # self.bn3 = nn.BatchNorm2d(32)
#         self.pool = nn.MaxPool2d(2,2)
#         self.fc1 = nn.Linear(32*53*53,128)
#         self.fc2 = nn.Linear(128,5)
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = self.relu(self.conv1(x))
#         x = self.pool(self.relu(self.conv2(x)))
#         x = self.pool(self.relu(self.conv3(x)))
#         x = x.view(-1, 32 * 53 * 53 )
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         return F.log_softmax(x, dim=1)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(Net())

20485

In [ ]:
from tqdm import tqdm
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    print(optimizer)
    net.train()
    for epoch in range(epochs):
      correct, total, epoch_loss = 0, 0, 0.0
      for images, labels in tqdm(trainloader):
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          optimizer.zero_grad()
          outputs = net(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          # Metrics
          epoch_loss += loss
          total += labels.size(0)
          correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
      epoch_loss /= len(trainloader.dataset)
      epoch_acc = correct / total
      if verbose:
          print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in tqdm(testloader):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
def print_data_split(trainloaders, valloaders):
    for i in range(len(trainloaders)):
        print("Client " + str(i+1) + " - Train Size: " + str(len(trainloaders[i].dataset)))
        print("Client " + str(i+1) + " - Test Size: " + str(len(valloaders[i].dataset)))

## Global Training (Without Federated Learning)

In [ ]:
train_global,val_global = split_dataset_for_global_training()

In [ ]:
net = Net().to(DEVICE)
for i in range(5):
    train(net, train_global, 1)
    loss_global, accuracy_global = test(net, val_global)
    print(f"Epoch {i+1}:  validation loss {loss_global}, accuracy {accuracy_global}")

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


100%|██████████| 92/92 [00:03<00:00, 23.06it/s]


Epoch 1:  validation loss 0.09373092698696696, accuracy 0.7431693989071039
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


100%|██████████| 92/92 [00:03<00:00, 25.72it/s]


Epoch 2:  validation loss 0.09099842527820194, accuracy 0.75
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


...

Epoch 3:  validation loss 0.09955016971448731, accuracy 0.7486338797814208
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


100%|██████████| 92/92 [00:03<00:00, 27.12it/s]


Epoch 4:  validation loss 0.08758724646775123, accuracy 0.7609289617486339
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


100%|██████████| 92/92 [00:03<00:00, 27.08it/s]

Epoch 5:  validation loss 0.09378358017388588, accuracy 0.7540983606557377


## Federated Learning

#### Utility Functions

In [ ]:
def get_parameters(net):
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = {k: torch.Tensor(v) for k, v in params_dict}
    net.load_state_dict(state_dict, strict=True)

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [ ]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [ ]:
def weighted_average(metrics):
    # Multiply accuracy of each client by number of examples used
    local_accuracies = [m["accuracy"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"global accuracy": sum(accuracies) / sum(examples), "local accuracies":local_accuracies}

# Can return individual if needed

In [ ]:
def run_fedAvg(num_of_rounds=5):
    # Create FedAvg strategy
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    # Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
    client_resources = None
    if DEVICE.type == "cuda":
        client_resources = {"num_gpus": 1}

    # Start simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=num_of_rounds),
        strategy=strategy,
        client_resources=client_resources,
    )

#### Starting federated learning with Number of Clients as 5 with strategy FedAvg

In [ ]:
NUM_CLIENTS = 5

#### Case 1: Equal Length Data & Unequal Classes

In [ ]:
trainloaders, valloaders = split_dataset_to_equal_length_but_unequal_classes(NUM_CLIENTS)

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 586
Client 1 - Test Size: 146
Client 2 - Train Size: 586
Client 2 - Test Size: 146
Client 3 - Train Size: 586
Client 3 - Test Size: 146
Client 4 - Train Size: 586
Client 4 - Test Size: 146
Client 5 - Train Size: 588
Client 5 - Test Size: 146


In [ ]:
run_fedAvg()

INFO flwr 2023-04-12 23:12:16,675 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-12 23:12:21,297	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-12 23:12:22,741 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3999812812.0, 'memory': 7999625627.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3999812812.0, 'memory': 7999625627.0, 'GPU': 1.0}
INFO flwr 2023-04-12 23:12:22,752 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-12 23:12:22,758 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=46056) Adam (
(launch_and_fit pid=46056) Parameter Group 0
(launch_and_fit pid=46056)     amsgrad: False
(launch_and_fit pid=46056)     betas: (0.9, 0.999)
(launch_and_fit pid=46056)     capturable: False
(launch_and_fit pid=46056)     differentiable: False
(launch_and_fit pid=46056)     eps: 1e-08
(launch_and_fit pid=46056)     foreach: None
(launch_and_fit pid=46056)     fused: None
(launch_and_fit pid=46056)     lr: 0.001
(launch_and_fit pid=46056)     maximize: False
(launch_and_fit pid=46056)     weight_decay: 0
(launch_and_fit pid=46056) )


100%|██████████| 74/74 [00:03<00:00, 24.38it/s]
(launch_and_fit pid=46153) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=46153)   warnings.warn(
(launch_and_fit pid=46153) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=46153)   warnings.warn(msg)
  0%|          | 0/74 [00:00<?, ?it/s]


(launch_and_fit pid=46153) Adam (
(launch_and_fit pid=46153) Parameter Group 0
(launch_and_fit pid=46153)     amsgrad: False
(launch_and_fit pid=46153)     betas: (0.9, 0.999)
(launch_and_fit pid=46153)     capturable: False
(launch_and_fit pid=46153)     differentiable: False
(launch_and_fit pid=46153)     eps: 1e-08
(launch_and_fit pid=46153)     foreach: None
(launch_and_fit pid=46153)     fused: None
(launch_and_fit pid=46153)     lr: 0.001
(launch_and_fit pid=46153)     maximize: False
(launch_and_fit pid=46153)     weight_decay: 0
(launch_and_fit pid=46153) )


100%|██████████| 74/74 [00:03<00:00, 20.16it/s]
(launch_and_fit pid=46251) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=46251)   warnings.warn(
(launch_and_fit pid=46251) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=46251)   warnings.warn(msg)


...

(launch_and_fit pid=50280) Adam (
(launch_and_fit pid=50280) Parameter Group 0
(launch_and_fit pid=50280)     amsgrad: False
(launch_and_fit pid=50280)     betas: (0.9, 0.999)
(launch_and_fit pid=50280)     capturable: False
(launch_and_fit pid=50280)     differentiable: False
(launch_and_fit pid=50280)     eps: 1e-08
(launch_and_fit pid=50280)     foreach: None
(launch_and_fit pid=50280)     fused: None
(launch_and_fit pid=50280)     lr: 0.001
(launch_and_fit pid=50280)     maximize: False
(launch_and_fit pid=50280)     weight_decay: 0
(launch_and_fit pid=50280) )


100%|██████████| 74/74 [00:04<00:00, 15.22it/s]
(launch_and_fit pid=50394) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=50394)   warnings.warn(
(launch_and_fit pid=50394) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=50394)   warnings.warn(msg)
  0%|          | 0/74 [00:00<?, ?it/s]


(launch_and_fit pid=50394) Adam (
(launch_and_fit pid=50394) Parameter Group 0
(launch_and_fit pid=50394)     amsgrad: False
(launch_and_fit pid=50394)     betas: (0.9, 0.999)
(launch_and_fit pid=50394)     capturable: False
(launch_and_fit pid=50394)     differentiable: False
(launch_and_fit pid=50394)     eps: 1e-08
(launch_and_fit pid=50394)     foreach: None
(launch_and_fit pid=50394)     fused: None
(launch_and_fit pid=50394)     lr: 0.001
(launch_and_fit pid=50394)     maximize: False
(launch_and_fit pid=50394)     weight_decay: 0
(launch_and_fit pid=50394) )


100%|██████████| 74/74 [00:03<00:00, 19.28it/s]
DEBUG flwr 2023-04-12 23:19:00,039 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-12 23:19:04,438 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)
(launch_and_evaluate pid=50534) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=50534)   warnings.warn(
(launch_and_evaluate pid=50534) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also 

History (loss, distributed):
	round 1: 0.10590127025258868
	round 2: 0.10066241909788079
	round 3: 0.10216906146235662
	round 4: 0.10789436837530708
	round 5: 0.11037223192110455
History (metrics, distributed):
{'global accuracy': [(1, 0.7123287671232876), (2, 0.7383561643835616), (3, 0.7232876712328766), (4, 0.6972602739726028), (5, 0.7027397260273972)], 'local accuracies': [(1, [0.7123287671232876, 0.7191780821917808, 0.7328767123287672, 0.7054794520547946, 0.6917808219178082]), (2, [0.7534246575342466, 0.7054794520547946, 0.7534246575342466, 0.7328767123287672, 0.7465753424657534]), (3, [0.7191780821917808, 0.7123287671232876, 0.7191780821917808, 0.7328767123287672, 0.7328767123287672]), (4, [0.7123287671232876, 0.6986301369863014, 0.7054794520547946, 0.6643835616438356, 0.7054794520547946]), (5, [0.7191780821917808, 0.6917808219178082, 0.7191780821917808, 0.6712328767123288, 0.7123287671232876])]}


#### Case 2: Equal Length Data & Equal Classes

In [ ]:
trainloaders, valloaders = data_splitter()

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 583
Client 1 - Test Size: 146
Client 2 - Train Size: 583
Client 2 - Test Size: 146
Client 3 - Train Size: 583
Client 3 - Test Size: 146
Client 4 - Train Size: 583
Client 4 - Test Size: 146
Client 5 - Train Size: 597
Client 5 - Test Size: 149


In [ ]:
run_fedAvg()

INFO flwr 2023-04-12 23:25:44,832 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-12 23:25:49,985	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-12 23:25:51,359 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3994667827.0, 'node:172.28.0.12': 1.0, 'memory': 7989335655.0, 'CPU': 2.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3994667827.0, 'node:172.28.0.12': 1.0, 'memory': 7989335655.0, 'CPU': 2.0, 'GPU': 1.0}
INFO flwr 2023-04-12 23:25:51,369 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-12 23:25:51,374 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=52878) Adam (
(launch_and_fit pid=52878) Parameter Group 0
(launch_and_fit pid=52878)     amsgrad: False
(launch_and_fit pid=52878)     betas: (0.9, 0.999)
(launch_and_fit pid=52878)     capturable: False
(launch_and_fit pid=52878)     differentiable: False
(launch_and_fit pid=52878)     eps: 1e-08
(launch_and_fit pid=52878)     foreach: None
(launch_and_fit pid=52878)     fused: None
(launch_and_fit pid=52878)     lr: 0.001
(launch_and_fit pid=52878)     maximize: False
(launch_and_fit pid=52878)     weight_decay: 0
(launch_and_fit pid=52878) )


100%|██████████| 73/73 [00:03<00:00, 18.46it/s]
(launch_and_fit pid=52981) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=52981)   warnings.warn(
(launch_and_fit pid=52981) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=52981)   warnings.warn(msg)
  0%|          | 0/73 [00:00<?, ?it/s]


(launch_and_fit pid=52981) Adam (
(launch_and_fit pid=52981) Parameter Group 0
(launch_and_fit pid=52981)     amsgrad: False
(launch_and_fit pid=52981)     betas: (0.9, 0.999)
(launch_and_fit pid=52981)     capturable: False
(launch_and_fit pid=52981)     differentiable: False
(launch_and_fit pid=52981)     eps: 1e-08
(launch_and_fit pid=52981)     foreach: None
(launch_and_fit pid=52981)     fused: None
(launch_and_fit pid=52981)     lr: 0.001
(launch_and_fit pid=52981)     maximize: False
(launch_and_fit pid=52981)     weight_decay: 0
(launch_and_fit pid=52981) )


100%|██████████| 73/73 [00:06<00:00, 11.92it/s]
(launch_and_fit pid=53097) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=53097)   warnings.warn(
(launch_and_fit pid=53097) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=53097)   warnings.warn(msg)
  0%|          | 0/73 [00:00<?, ?it/s]


...

(launch_and_fit pid=57274) Adam (
(launch_and_fit pid=57274) Parameter Group 0
(launch_and_fit pid=57274)     amsgrad: False
(launch_and_fit pid=57274)     betas: (0.9, 0.999)
(launch_and_fit pid=57274)     capturable: False
(launch_and_fit pid=57274)     differentiable: False
(launch_and_fit pid=57274)     eps: 1e-08
(launch_and_fit pid=57274)     foreach: None
(launch_and_fit pid=57274)     fused: None
(launch_and_fit pid=57274)     lr: 0.001
(launch_and_fit pid=57274)     maximize: False
(launch_and_fit pid=57274)     weight_decay: 0
(launch_and_fit pid=57274) )


100%|██████████| 73/73 [00:04<00:00, 17.25it/s]
DEBUG flwr 2023-04-12 23:32:39,343 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-12 23:32:46,332 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)
(raylet) [2023-04-12 23:32:49,912 E 52632 52632] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: a230ee0bf254e1d276b41199811fde2a7ba0e55236e93448c09508ea, IP: 172.28.0.12) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.0.12`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or red

(scheduler +31m40s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(launch_and_evaluate pid=57632) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=57632)   warnings.warn(
(launch_and_evaluate pid=57632) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_evaluate pid=57632)   warnings.warn(msg)
100%|██████████| 19/19 [00:01<00:00, 12.03it/s]
(launch_and_evaluate pid=57705) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, p

History (loss, distributed):
	round 1: 0.09792609589825642
	round 2: 0.09911939934905732
	round 3: 0.08991503474071665
	round 4: 0.09606406101370445
	round 5: 0.09524589771759515
History (metrics, distributed):
{'global accuracy': [(1, 0.7450491863565321), (2, 0.7149949434586743), (3, 0.7422267169256228), (4, 0.7068309276454905), (5, 0.7422542980601269)], 'local accuracies': [(1, [0.7465753424657534, 0.8013698630136986, 0.7046979865771812, 0.7397260273972602, 0.7328767123287672]), (2, [0.7328767123287672, 0.684931506849315, 0.6845637583892618, 0.7602739726027398, 0.7123287671232876]), (3, [0.7534246575342466, 0.7397260273972602, 0.7808219178082192, 0.7248322147651006, 0.7123287671232876]), (4, [0.7123287671232876, 0.7397260273972602, 0.7123287671232876, 0.6711409395973155, 0.6986301369863014]), (5, [0.7181208053691275, 0.726027397260274, 0.7465753424657534, 0.7328767123287672, 0.7876712328767124])]}

#### Case 3: Unequal Length Data & Unequal Classes

In [ ]:
trainloaders, valloaders=data_splitter([0.3,0.5,0.1,0.05,0.05],[0.1,0.3,0.2,0.2,0.2],[0.05,0.2,0.15,0.5,0.1],[0.1,0.1,0.25,0.35,0.2],[0.3,0.1,0.1,0.1,0.4])

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 586
Client 1 - Test Size: 146
Client 2 - Train Size: 1007
Client 2 - Test Size: 146
Client 3 - Train Size: 383
Client 3 - Test Size: 146
Client 4 - Train Size: 606
Client 4 - Test Size: 146
Client 5 - Train Size: 347
Client 5 - Test Size: 149


In [ ]:
run_fedAvg()

INFO flwr 2023-04-12 23:44:34,563 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-12 23:44:39,963	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-12 23:44:41,405 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'memory': 7998404199.0, 'object_store_memory': 3999202099.0, 'GPU': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'memory': 7998404199.0, 'object_store_memory': 3999202099.0, 'GPU': 1.0, 'CPU': 2.0}
INFO flwr 2023-04-12 23:44:41,416 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-12 23:44:41,420 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=61189) Adam (
(launch_and_fit pid=61189) Parameter Group 0
(launch_and_fit pid=61189)     amsgrad: False
(launch_and_fit pid=61189)     betas: (0.9, 0.999)
(launch_and_fit pid=61189)     capturable: False
(launch_and_fit pid=61189)     differentiable: False
(launch_and_fit pid=61189)     eps: 1e-08
(launch_and_fit pid=61189)     foreach: None
(launch_and_fit pid=61189)     fused: None
(launch_and_fit pid=61189)     lr: 0.001
(launch_and_fit pid=61189)     maximize: False
(launch_and_fit pid=61189)     weight_decay: 0
(launch_and_fit pid=61189) )


 12%|█▏        | 15/126 [00:00<00:06, 18.39it/s]


(scheduler +43m23s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


100%|██████████| 126/126 [00:06<00:00, 19.17it/s]
(launch_and_fit pid=61322) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=61322)   warnings.warn(
(launch_and_fit pid=61322) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=61322)   warnings.warn(msg)
  0%|          | 0/74 [00:00<?, ?it/s]


...

100%|██████████| 74/74 [00:04<00:00, 16.03it/s]
(launch_and_fit pid=65558) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=65558)   warnings.warn(
(launch_and_fit pid=65558) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=65558)   warnings.warn(msg)
  0%|          | 0/48 [00:00<?, ?it/s]


(launch_and_fit pid=65558) Adam (
(launch_and_fit pid=65558) Parameter Group 0
(launch_and_fit pid=65558)     amsgrad: False
(launch_and_fit pid=65558)     betas: (0.9, 0.999)
(launch_and_fit pid=65558)     capturable: False
(launch_and_fit pid=65558)     differentiable: False
(launch_and_fit pid=65558)     eps: 1e-08
(launch_and_fit pid=65558)     foreach: None
(launch_and_fit pid=65558)     fused: None
(launch_and_fit pid=65558)     lr: 0.001
(launch_and_fit pid=65558)     maximize: False
(launch_and_fit pid=65558)     weight_decay: 0
(launch_and_fit pid=65558) )


100%|██████████| 48/48 [00:02<00:00, 20.77it/s]
DEBUG flwr 2023-04-12 23:51:22,216 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-12 23:51:27,790 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)


(scheduler +50m5s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(raylet) [2023-04-12 23:51:39,882 E 60943 60943] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 172302d6824b6cc7d2f9fb762cb80d5e5a4dcfdf993e185848f93f25, IP: 172.28.0.12) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.0.12`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(launch_and_evaluate pid=65681) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: T

#### Case 4: Unequal Classes
##### Case I: 1 class per client

In [ ]:
trainloaders, valloaders=data_splitter([0.99,0.0025,0.0025,0.0025,0.0025],[0.0025,0.99,0.0025,0.0025,0.0025],[0.0025,0.0025,0.99,0.0025,0.0025],[0.0025,0.0025,0.0025,0.99,0.0025],[0.0025,0.0025,0.0025,0.0025,0.99])

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 1430
Client 1 - Test Size: 146
Client 2 - Train Size: 297
Client 2 - Test Size: 146
Client 3 - Train Size: 794
Client 3 - Test Size: 146
Client 4 - Train Size: 156
Client 4 - Test Size: 146
Client 5 - Train Size: 252
Client 5 - Test Size: 149


In [ ]:
run_fedAvg()

INFO flwr 2023-04-12 23:59:42,020 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-12 23:59:47,523	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-12 23:59:48,926 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3999790694.0, 'node:172.28.0.12': 1.0, 'memory': 7999581390.0, 'GPU': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3999790694.0, 'node:172.28.0.12': 1.0, 'memory': 7999581390.0, 'GPU': 1.0, 'CPU': 2.0}
INFO flwr 2023-04-12 23:59:48,934 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-12 23:59:48,940 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=68436) Adam (
(launch_and_fit pid=68436) Parameter Group 0
(launch_and_fit pid=68436)     amsgrad: False
(launch_and_fit pid=68436)     betas: (0.9, 0.999)
(launch_and_fit pid=68436)     capturable: False
(launch_and_fit pid=68436)     differentiable: False
(launch_and_fit pid=68436)     eps: 1e-08
(launch_and_fit pid=68436)     foreach: None
(launch_and_fit pid=68436)     fused: None
(launch_and_fit pid=68436)     lr: 0.001
(launch_and_fit pid=68436)     maximize: False
(launch_and_fit pid=68436)     weight_decay: 0
(launch_and_fit pid=68436) )


100%|██████████| 38/38 [00:02<00:00, 13.84it/s]
(launch_and_fit pid=68528) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=68528)   warnings.warn(
(launch_and_fit pid=68528) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=68528)   warnings.warn(msg)


(launch_and_fit pid=68528) Adam (
(launch_and_fit pid=68528) Parameter Group 0
(launch_and_fit pid=68528)     amsgrad: False
(launch_and_fit pid=68528)     betas: (0.9, 0.999)
(launch_and_fit pid=68528)     capturable: False
(launch_and_fit pid=68528)     differentiable: False
(launch_and_fit pid=68528)     eps: 1e-08
(launch_and_fit pid=68528)     foreach: None
(launch_and_fit pid=68528)     fused: None
(launch_and_fit pid=68528)     lr: 0.001
(launch_and_fit pid=68528)     maximize: False
(launch_and_fit pid=68528)     weight_decay: 0
(launch_and_fit pid=68528) )


100%|██████████| 20/20 [00:01<00:00, 16.97it/s]
(launch_and_fit pid=68605) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=68605)   warnings.warn(
(launch_and_fit pid=68605) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=68605)   warnings.warn(msg)


...

100%|██████████| 20/20 [00:01<00:00, 18.37it/s]
(launch_and_fit pid=72875) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=72875)   warnings.warn(
(launch_and_fit pid=72875) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=72875)   warnings.warn(msg)
  0%|          | 0/32 [00:00<?, ?it/s]


(launch_and_fit pid=72875) Adam (
(launch_and_fit pid=72875) Parameter Group 0
(launch_and_fit pid=72875)     amsgrad: False
(launch_and_fit pid=72875)     betas: (0.9, 0.999)
(launch_and_fit pid=72875)     capturable: False
(launch_and_fit pid=72875)     differentiable: False
(launch_and_fit pid=72875)     eps: 1e-08
(launch_and_fit pid=72875)     foreach: None
(launch_and_fit pid=72875)     fused: None
(launch_and_fit pid=72875)     lr: 0.001
(launch_and_fit pid=72875)     maximize: False
(launch_and_fit pid=72875)     weight_decay: 0
(launch_and_fit pid=72875) )


100%|██████████| 32/32 [00:01<00:00, 20.38it/s]
DEBUG flwr 2023-04-13 00:06:32,121 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-13 00:06:37,595 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)
(raylet) [2023-04-13 00:06:47,432 E 68204 68204] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 7dc1fe68b82eb7179eca7660876a85fa4fbde6c4b8158aa3fbc4d130, IP: 172.28.0.12) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.0.12`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or red

(scheduler +1h5m22s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(launch_and_evaluate pid=72988) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=72988)   warnings.warn(
(launch_and_evaluate pid=72988) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_evaluate pid=72988)   warnings.warn(msg)
100%|██████████| 19/19 [00:02<00:00,  8.37it/s]
(launch_and_evaluate pid=73118) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, p

##### Case II: 2 class per client

In [ ]:
trainloaders, valloaders=data_splitter([0.49,0.01,0.01,0.01,0.48],[0.48,0.49,0.01,0.01,0.01],[0.01,0.48,0.49,0.01,0.01],[0.01,0.01,0.48,0.49,0.01],[0.01,0.01,0.01,0.48,0.49])

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 859
Client 1 - Test Size: 146
Client 2 - Train Size: 545
Client 2 - Test Size: 146
Client 3 - Train Size: 482
Client 3 - Test Size: 146
Client 4 - Train Size: 211
Client 4 - Test Size: 146
Client 5 - Train Size: 832
Client 5 - Test Size: 149


In [ ]:
run_fedAvg()

INFO flwr 2023-04-13 00:55:38,647 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-13 00:55:40,045	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-13 00:55:41,499 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'memory': 7929016320.0, 'object_store_memory': 3964508160.0, 'GPU': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'memory': 7929016320.0, 'object_store_memory': 3964508160.0, 'GPU': 1.0, 'CPU': 2.0}
INFO flwr 2023-04-13 00:55:41,507 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-13 00:55:41,520 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=4089) Adam (
(launch_and_fit pid=4089) Parameter Group 0
(launch_and_fit pid=4089)     amsgrad: False
(launch_and_fit pid=4089)     betas: (0.9, 0.999)
(launch_and_fit pid=4089)     capturable: False
(launch_and_fit pid=4089)     differentiable: False
(launch_and_fit pid=4089)     eps: 1e-08
(launch_and_fit pid=4089)     foreach: None
(launch_and_fit pid=4089)     fused: None
(launch_and_fit pid=4089)     lr: 0.001
(launch_and_fit pid=4089)     maximize: False
(launch_and_fit pid=4089)     weight_decay: 0
(launch_and_fit pid=4089) )


100%|██████████| 27/27 [00:40<00:00,  1.52s/it]
(launch_and_fit pid=4380) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=4380)   warnings.warn(
(launch_and_fit pid=4380) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=4380)   warnings.warn(msg)
  0%|          | 0/104 [00:00<?, ?it/s]


(launch_and_fit pid=4380) Adam (
(launch_and_fit pid=4380) Parameter Group 0
(launch_and_fit pid=4380)     amsgrad: False
(launch_and_fit pid=4380)     betas: (0.9, 0.999)
(launch_and_fit pid=4380)     capturable: False
(launch_and_fit pid=4380)     differentiable: False
(launch_and_fit pid=4380)     eps: 1e-08
(launch_and_fit pid=4380)     foreach: None
(launch_and_fit pid=4380)     fused: None
(launch_and_fit pid=4380)     lr: 0.001
(launch_and_fit pid=4380)     maximize: False
(launch_and_fit pid=4380)     weight_decay: 0
(launch_and_fit pid=4380) )


100%|██████████| 104/104 [04:05<00:00,  2.36s/it]
(launch_and_fit pid=5641) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=5641)   warnings.warn(
(launch_and_fit pid=5641) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=5641)   warnings.warn(msg)
  0%|          | 0/61 [00:00<?, ?it/s]


...

100%|██████████| 27/27 [00:01<00:00, 15.73it/s]
(launch_and_fit pid=13154) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=13154)   warnings.warn(
(launch_and_fit pid=13154) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=13154)   warnings.warn(msg)
  0%|          | 0/69 [00:00<?, ?it/s]


(launch_and_fit pid=13154) Adam (
(launch_and_fit pid=13154) Parameter Group 0
(launch_and_fit pid=13154)     amsgrad: False
(launch_and_fit pid=13154)     betas: (0.9, 0.999)
(launch_and_fit pid=13154)     capturable: False
(launch_and_fit pid=13154)     differentiable: False
(launch_and_fit pid=13154)     eps: 1e-08
(launch_and_fit pid=13154)     foreach: None
(launch_and_fit pid=13154)     fused: None
(launch_and_fit pid=13154)     lr: 0.001
(launch_and_fit pid=13154)     maximize: False
(launch_and_fit pid=13154)     weight_decay: 0
(launch_and_fit pid=13154) )


100%|██████████| 69/69 [00:04<00:00, 14.68it/s]
(launch_and_fit pid=13266) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=13266)   warnings.warn(
(launch_and_fit pid=13266) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=13266)   warnings.warn(msg)


(launch_and_fit pid=13266) Adam (
(launch_and_fit pid=13266) Parameter Group 0
(launch_and_fit pid=13266)     amsgrad: False
(launch_and_fit pid=13266)     betas: (0.9, 0.999)
(launch_and_fit pid=13266)     capturable: False
(launch_and_fit pid=13266)     differentiable: False
(launch_and_fit pid=13266)     eps: 1e-08
(launch_and_fit pid=13266)     foreach: None
(launch_and_fit pid=13266)     fused: None
(launch_and_fit pid=13266)     lr: 0.001
(launch_and_fit pid=13266)     maximize: False
(launch_and_fit pid=13266)     weight_decay: 0
(launch_and_fit pid=13266) )


100%|██████████| 104/104 [00:05<00:00, 18.85it/s]
DEBUG flwr 2023-04-13 01:19:17,082 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-13 01:19:21,342 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)
(raylet) Spilled 8285 MiB, 84 objects, write throughput 233 MiB/s.
(launch_and_evaluate pid=13427) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=13427)   warnings.warn(
(launch_and_evaluate pid=13427) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equival

##### Case III: 3 class per client

In [ ]:
trainloaders, valloaders=data_splitter([0.33,0.005,0.005,0.33,0.33],[0.33,0.33,0.005,0.005,0.33],[0.33,0.33,0.33,0.005,0.005],[0.005,0.33,0.33,0.33,0.005],[0.005,0.005,0.33,0.33,0.33])

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 837
Client 1 - Test Size: 146
Client 2 - Train Size: 418
Client 2 - Test Size: 146
Client 3 - Train Size: 398
Client 3 - Test Size: 146
Client 4 - Train Size: 607
Client 4 - Test Size: 146
Client 5 - Train Size: 669
Client 5 - Test Size: 149


In [ ]:
run_fedAvg()

INFO flwr 2023-04-16 22:34:56,229 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-16 22:34:58,666	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-16 22:35:00,047 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3959054745.0, 'memory': 7918109492.0, 'CPU': 2.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3959054745.0, 'memory': 7918109492.0, 'CPU': 2.0, 'GPU': 1.0}
INFO flwr 2023-04-16 22:35:00,055 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-16 22:35:00,064 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=20689) Adam (
(launch_and_fit pid=20689) Parameter Group 0
(launch_and_fit pid=20689)     amsgrad: False
(launch_and_fit pid=20689)     betas: (0.9, 0.999)
(launch_and_fit pid=20689)     capturable: False
(launch_and_fit pid=20689)     differentiable: False
(launch_and_fit pid=20689)     eps: 1e-08
(launch_and_fit pid=20689)     foreach: None
(launch_and_fit pid=20689)     fused: None
(launch_and_fit pid=20689)     lr: 0.001
(launch_and_fit pid=20689)     maximize: False
(launch_and_fit pid=20689)     weight_decay: 0
(launch_and_fit pid=20689) )


100%|██████████| 105/105 [00:04<00:00, 24.34it/s]
(launch_and_fit pid=20809) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=20809)   warnings.warn(
(launch_and_fit pid=20809) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=20809)   warnings.warn(msg)


(launch_and_fit pid=20809) Adam (
(launch_and_fit pid=20809) Parameter Group 0
(launch_and_fit pid=20809)     amsgrad: False
(launch_and_fit pid=20809)     betas: (0.9, 0.999)
(launch_and_fit pid=20809)     capturable: False
(launch_and_fit pid=20809)     differentiable: False
(launch_and_fit pid=20809)     eps: 1e-08
(launch_and_fit pid=20809)     foreach: None
(launch_and_fit pid=20809)     fused: None
(launch_and_fit pid=20809)     lr: 0.001
(launch_and_fit pid=20809)     maximize: False
(launch_and_fit pid=20809)     weight_decay: 0
(launch_and_fit pid=20809) )


100%|██████████| 84/84 [00:04<00:00, 20.73it/s]
(launch_and_fit pid=20920) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=20920)   warnings.warn(
(launch_and_fit pid=20920) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=20920)   warnings.warn(msg)


...

100%|██████████| 53/53 [00:02<00:00, 17.82it/s]
(launch_and_fit pid=24885) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=24885)   warnings.warn(
(launch_and_fit pid=24885) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=24885)   warnings.warn(msg)
  0%|          | 0/105 [00:00<?, ?it/s]


(launch_and_fit pid=24885) Adam (
(launch_and_fit pid=24885) Parameter Group 0
(launch_and_fit pid=24885)     amsgrad: False
(launch_and_fit pid=24885)     betas: (0.9, 0.999)
(launch_and_fit pid=24885)     capturable: False
(launch_and_fit pid=24885)     differentiable: False
(launch_and_fit pid=24885)     eps: 1e-08
(launch_and_fit pid=24885)     foreach: None
(launch_and_fit pid=24885)     fused: None
(launch_and_fit pid=24885)     lr: 0.001
(launch_and_fit pid=24885)     maximize: False
(launch_and_fit pid=24885)     weight_decay: 0
(launch_and_fit pid=24885) )


100%|██████████| 105/105 [00:06<00:00, 15.94it/s]
(launch_and_fit pid=25025) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=25025)   warnings.warn(
(launch_and_fit pid=25025) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=25025)   warnings.warn(msg)
  0%|          | 0/84 [00:00<?, ?it/s]


(launch_and_fit pid=25025) Adam (
(launch_and_fit pid=25025) Parameter Group 0
(launch_and_fit pid=25025)     amsgrad: False
(launch_and_fit pid=25025)     betas: (0.9, 0.999)
(launch_and_fit pid=25025)     capturable: False
(launch_and_fit pid=25025)     differentiable: False
(launch_and_fit pid=25025)     eps: 1e-08
(launch_and_fit pid=25025)     foreach: None
(launch_and_fit pid=25025)     fused: None
(launch_and_fit pid=25025)     lr: 0.001
(launch_and_fit pid=25025)     maximize: False
(launch_and_fit pid=25025)     weight_decay: 0
(launch_and_fit pid=25025) )


100%|██████████| 84/84 [00:04<00:00, 20.13it/s]
DEBUG flwr 2023-04-16 22:41:38,216 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-16 22:41:43,155 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)
(launch_and_evaluate pid=25162) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=25162)   warnings.warn(
(launch_and_evaluate pid=25162) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also 

##### Case IV: 4 class per client

In [ ]:
trainloaders, valloaders=data_splitter([0.2499,0.0004,0.2499,0.2499,0.2499],[0.2499,0.2499,0.0004,0.2499,0.2499],[0.2499,0.2499,0.2499,0.0004,0.2499],[0.2499,0.2499,0.2499,0.2499,0.0004],[0.0004,0.2499,0.2499,0.2499,0.2499])

In [ ]:
print_data_split(trainloaders, valloaders)

Client 1 - Train Size: 670
Client 1 - Test Size: 146
Client 2 - Train Size: 368
Client 2 - Test Size: 146
Client 3 - Train Size: 655
Client 3 - Test Size: 146
Client 4 - Train Size: 529
Client 4 - Test Size: 146
Client 5 - Train Size: 707
Client 5 - Test Size: 149


In [ ]:
run_fedAvg()

INFO flwr 2023-04-13 01:55:46,668 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-13 01:55:52,017	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-13 01:55:53,397 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 4000727040.0, 'memory': 8001454080.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 4000727040.0, 'memory': 8001454080.0, 'GPU': 1.0}
INFO flwr 2023-04-13 01:55:53,401 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-13 01:55:53,407 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_fit pid=30190) Adam (
(launch_and_fit pid=30190) Parameter Group 0
(launch_and_fit pid=30190)     amsgrad: False
(launch_and_fit pid=30190)     betas: (0.9, 0.999)
(launch_and_fit pid=30190)     capturable: False
(launch_and_fit pid=30190)     differentiable: False
(launch_and_fit pid=30190)     eps: 1e-08
(launch_and_fit pid=30190)     foreach: None
(launch_and_fit pid=30190)     fused: None
(launch_and_fit pid=30190)     lr: 0.001
(launch_and_fit pid=30190)     maximize: False
(launch_and_fit pid=30190)     weight_decay: 0
(launch_and_fit pid=30190) )


100%|██████████| 89/89 [00:05<00:00, 17.15it/s]
(launch_and_fit pid=30310) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=30310)   warnings.warn(
(launch_and_fit pid=30310) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=30310)   warnings.warn(msg)


(launch_and_fit pid=30310) Adam (
(launch_and_fit pid=30310) Parameter Group 0
(launch_and_fit pid=30310)     amsgrad: False
(launch_and_fit pid=30310)     betas: (0.9, 0.999)
(launch_and_fit pid=30310)     capturable: False
(launch_and_fit pid=30310)     differentiable: False
(launch_and_fit pid=30310)     eps: 1e-08
(launch_and_fit pid=30310)     foreach: None
(launch_and_fit pid=30310)     fused: None
(launch_and_fit pid=30310)     lr: 0.001
(launch_and_fit pid=30310)     maximize: False
(launch_and_fit pid=30310)     weight_decay: 0
(launch_and_fit pid=30310) )


100%|██████████| 84/84 [00:05<00:00, 16.63it/s]
(launch_and_fit pid=30433) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=30433)   warnings.warn(
(launch_and_fit pid=30433) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=30433)   warnings.warn(msg)


...

100%|██████████| 82/82 [00:04<00:00, 19.00it/s]
(launch_and_fit pid=34519) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=34519)   warnings.warn(
(launch_and_fit pid=34519) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=34519)   warnings.warn(msg)


(launch_and_fit pid=34519) Adam (
(launch_and_fit pid=34519) Parameter Group 0
(launch_and_fit pid=34519)     amsgrad: False
(launch_and_fit pid=34519)     betas: (0.9, 0.999)
(launch_and_fit pid=34519)     capturable: False
(launch_and_fit pid=34519)     differentiable: False
(launch_and_fit pid=34519)     eps: 1e-08
(launch_and_fit pid=34519)     foreach: None
(launch_and_fit pid=34519)     fused: None
(launch_and_fit pid=34519)     lr: 0.001
(launch_and_fit pid=34519)     maximize: False
(launch_and_fit pid=34519)     weight_decay: 0
(launch_and_fit pid=34519) )


100%|██████████| 67/67 [00:04<00:00, 14.66it/s]
(launch_and_fit pid=34635) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=34635)   warnings.warn(
(launch_and_fit pid=34635) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=34635)   warnings.warn(msg)
  0%|          | 0/89 [00:00<?, ?it/s]


(launch_and_fit pid=34635) Adam (
(launch_and_fit pid=34635) Parameter Group 0
(launch_and_fit pid=34635)     amsgrad: False
(launch_and_fit pid=34635)     betas: (0.9, 0.999)
(launch_and_fit pid=34635)     capturable: False
(launch_and_fit pid=34635)     differentiable: False
(launch_and_fit pid=34635)     eps: 1e-08
(launch_and_fit pid=34635)     foreach: None
(launch_and_fit pid=34635)     fused: None
(launch_and_fit pid=34635)     lr: 0.001
(launch_and_fit pid=34635)     maximize: False
(launch_and_fit pid=34635)     weight_decay: 0
(launch_and_fit pid=34635) )


100%|██████████| 89/89 [00:04<00:00, 20.34it/s]
DEBUG flwr 2023-04-13 02:02:41,755 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG:flwr:fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-04-13 02:02:47,223 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 5)
DEBUG:flwr:evaluate_round 5: strategy sampled 5 clients (out of 5)
(launch_and_evaluate pid=34778) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=34778)   warnings.warn(
(launch_and_evaluate pid=34778) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also 

In [ ]:
# trainloaders, valloaders=data_splitter([0.99,0.0025,0.0025,0.0025,0.0025],[0.0025,0.99,0.0025,0.0025,0.0025],[0.0025,0.0025,0.99,0.0025,0.0025],[0.0025,0.0025,0.0025,0.99,0.0025],[0.0025,0.0025,0.0025,0.0025,0.99])
# trainloaders, valloaders = data_splitter()

In [ ]:
# Create FedProx strategy
# strategy = fl.server.strategy.FedProx(
#     fraction_fit=1.0,
#     evaluate_metrics_aggregation_fn=weighted_average,
#     proximal_mu=0.8,  # <-- pass the metric aggregation function
# )

# # Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
# client_resources = None
# if DEVICE.type == "cuda":
#     client_resources = {"num_gpus": 1}

# # Start simulation
# fl.simulation.start_simulation(
#     client_fn=client_fn,
#     num_clients=NUM_CLIENTS,
#     config=fl.server.ServerConfig(num_rounds=5),
#     strategy=strategy,
#     client_resources=client_resources,
# )